# Overview Description
this file is the final file to create a regression model to observe the relationship between customer behavior and their profitability. after merging the result of clustering and features data, we are going to implement the StandardScaler to Normalize the values of features to avoid the features domination of model. then, the logistic regression would be the estimator for recursive feature elimination to select the most important features from the model. At last, the regression model with the best features would be run again to show the coefficient of features and confusion matrix to observe the misclassification rate.

In [11]:
import pandas as pd
cluster_result = pd.read_csv(r"C:\Users\joech\OneDrive\workspace\HKTV OpenDataBank\RFM_project\official_draft\dataset\cluster_result.csv", index_col = 0)
input_features = pd.read_csv(r"C:\Users\joech\OneDrive\workspace\HKTV OpenDataBank\RFM_project\official_draft\dataset\membership_level_aggregation.csv", index_col = 0)
dataset = cluster_result.merge(
    input_features,
    how="left",
    left_index = True,
    right_index = True
)
dataset.rename({"0":"is_profitable"}, inplace=True, axis=1)

In [12]:
target_dataset = dataset.copy()
target_dataset.drop(['total_order_value', 'order_number_count', "app_type_LITE", "app_type_MAIN", "items_purchased"], inplace=True, axis=1)
# since the RFM score is made from total order value and order number count, therefore, we are going to elimilate the columns

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import numpy as np

ss=StandardScaler()
features = ss.fit_transform(X=target_dataset.drop("is_profitable", axis=1))
target = target_dataset.is_profitable

In [14]:
estimator = LogisticRegression(random_state=0, penalty = "l1", solver = "liblinear")
best = RFE(estimator, n_features_to_select=10, step=0.05).fit(features, target)
columns = target_dataset.drop("is_profitable", axis=1).columns[best.support_]
model = LogisticRegression(random_state = 0, penalty="l1", solver = "liblinear").fit(features[:,best.support_], target) #the lasso regression would be performed because I want it drop the columns to
pd.DataFrame(model.coef_, columns=columns).T.sort_values(by=0)

,0
prim_cat_variety,-2.874463
sub_cat_4_variety,0.310319
primary_category_name_en_Mother & Baby,0.359407
total_package_weight,0.366072
order_date_weekdays_Wed,0.454174
primary_category_name_en_Housewares,0.533115
primary_category_name_en_Personal Care & Health,0.614956
items_under_promotion,0.663664
sales_application_Web,2.827818
sales_application_WebMobile,3.585540


In [15]:
predict = model.predict(features[:,best.support_])
confusion_matrix(target, predict)

array([[9348,  442],
       [1284, 1023]], dtype=int64)